# Wandb for tracking

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.9 MB 13.2 MB/s 
     |████████████████████████████████| 182 kB 63.2 MB/s 
     |████████████████████████████████| 168 kB 11.8 MB/s 
     |████████████████████████████████| 62 kB 733 kB/s 
     |████████████████████████████████| 168 kB 32.6 MB/s 
     |████████████████████████████████| 166 kB 68.0 MB/s 
     |████████████████████████████████| 166 kB 70.1 MB/s 
     |████████████████████████████████| 162 kB 66.0 MB/s 
     |████████████████████████████████| 162 kB 66.3 MB/s 
     |████████████████████████████████| 158 kB 26.9 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 11.3 MB/s 
     |████████████████████████████████| 157 kB 64.7 MB/s 
     |████████████████████████████████| 157 kB 96.4 MB/s 
     |████████████████████████████████| 157 kB 83.8 MB/s 
     |████████████████████████████████| 157 kB 49.0 MB/s 
     |████████████████████████████████| 157 kB 57.5 MB/s 
     |██████████

In [2]:
import wandb

In [3]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%env WANDB_PROJECT=bert-base-ner

env: WANDB_PROJECT=bert-base-ner


# Data downloader


In [5]:
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install evaluate -q
!pip install datasets -q

     |████████████████████████████████| 7.6 MB 14.7 MB/s 
     |████████████████████████████████| 5.8 MB 14.9 MB/s 
     |████████████████████████████████| 182 kB 72.0 MB/s 
     |████████████████████████████████| 43 kB 951 kB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 451 kB 32.2 MB/s 
     |████████████████████████████████| 212 kB 60.3 MB/s 
     |████████████████████████████████| 132 kB 8.2 MB/s 
     |████████████████████████████████| 127 kB 68.5 MB/s 


In [6]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
labels_names = dataset["train"].features["ner_tags"].feature.names

In [8]:
labels_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
dataset["train"][:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

# Data preparing

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding = "max_length", truncation = True, is_split_into_words = True)

In [12]:
tokenized_datasets_ = dataset.map(tokenize_function, batched = True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
len(tokenized_datasets_["train"][0]["input_ids"]) == len(tokenized_datasets_["train"][0]["ner_tags"])

False

Adjusting labels to fit input

In [14]:
def tokenize_adjust_labels(samples):
    tokenized_samples = tokenizer.batch_encode_plus(samples["tokens"], is_split_into_words = True, truncation = True)

    total_adjusted_labels = []

    for k in range(len(tokenized_samples["input_ids"])):
        prev_wid = -1

        word_ids_list = tokenized_samples.word_ids(batch_index = k)

        existing_label_ids = samples["ner_tags"][k]

        i = -1

        adjusted_label_ids = []

        for word_idx in word_ids_list:
            if word_idx is None:
                adjusted_label_ids.append(-100)
            elif word_idx != prev_wid:
                i += 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = word_idx
            else:
                label_name = labels_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])
        
        total_adjusted_labels.append(adjusted_label_ids)
            
    tokenized_samples["labels"] = total_adjusted_labels

    return tokenized_samples


In [15]:
out = tokenizer("Fine tune NER with BERT")
out

{'input_ids': [101, 2986, 8694, 11265, 2099, 2007, 14324, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
out.word_ids(0)

[None, 0, 1, 2, 2, 3, 4, None]

In [17]:
tokenized_dataset = dataset.map(tokenize_adjust_labels, batched = True, remove_columns = ["tokens", "ner_tags", "pos_tags", "chunk_tags"])

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [19]:
tokenized_dataset["train"][:2]

{'id': ['0', '1'],
 'input_ids': [[101,
   7327,
   19164,
   2446,
   2655,
   2000,
   17757,
   2329,
   12559,
   1012,
   102],
  [101, 2848, 13934, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100]]}

Padding token to a constant length

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

# Fine-tuning BERT for NER task

In [21]:
from transformers import AutoModelForTokenClassification

In [22]:
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = len(labels_names))

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

## Computing metrics 

In [23]:
import numpy as np
import evaluate

In [24]:
metric = evaluate.load("seqeval")

In [25]:
def compute_metrics(p):
    predictions, labels = p

    predictions = np.argmax(predictions, axis = 2)

    true_predictions = [
        [labels_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [labels_names[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions = true_predictions, references = true_labels)

    return {
        "precision" : results["overall_precision"],
        "recall" : results["overall_recall"],
        "f1" : results["overall_f1"],
        "accuracy" : results["overall_accuracy"]
    }

## Using Trainer API to fine-tune

In [26]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [27]:
batch_size = 16
logging_steps = len(tokenized_dataset["train"]) // batch_size
epochs = 7
weight_decay = .01
eval_steps = 100
learning_rate = 2e-5
early_stopping_patience = 3

In [28]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    learning_rate = learning_rate,
    logging_steps=logging_steps,
    evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = eval_steps,
    save_total_limit = 5,
    metric_for_best_model = "eval_loss",
    load_best_model_at_end = True,
    report_to = "wandb"
) 

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)

In [ ]:
trainer.train()

## Testing

In [31]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [
    [labels_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [labels_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id. If id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3453
  Batch size = 16


In [32]:
results

{'LOC': {'precision': 0.9059101654846335,
  'recall': 0.9020715630885122,
  'f1': 0.9039867893371079,
  'number': 2124},
 'MISC': {'precision': 0.7630662020905923,
  'recall': 0.6596385542168675,
  'f1': 0.7075928917609047,
  'number': 996},
 'ORG': {'precision': 0.8422782037239869,
  'recall': 0.8914219474497682,
  'f1': 0.8661535573493524,
  'number': 2588},
 'PER': {'precision': 0.9575735020468924,
  'recall': 0.9466519499632082,
  'f1': 0.9520814061054578,
  'number': 2718},
 'overall_precision': 0.8870507022137586,
 'overall_recall': 0.8845240920958937,
 'overall_f1': 0.8857855954361779,
 'overall_accuracy': 0.973586078309509}

In [33]:
true_predictions

[['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
 

In [34]:
true_labels

[['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',


In [35]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [36]:
y_pred = []
y = []
for sent in true_predictions:
    for tag in sent:
        y_pred.append(tag)
for sent in true_labels:
    for tag in sent:
        y.append(tag)


In [37]:
print(classification_report(y_pred, y))

              precision    recall  f1-score   support

       B-LOC       0.91      0.92      0.91      2100
      B-MISC       0.69      0.82      0.75       840
       B-ORG       0.91      0.87      0.89      2695
       B-PER       0.95      0.96      0.96      2680
       I-LOC       0.84      0.82      0.83       284
      I-MISC       0.64      0.62      0.63       248
       I-ORG       0.90      0.81      0.85      1166
       I-PER       0.99      0.98      0.99      2530
           O       0.99      0.99      0.99     45381

    accuracy                           0.97     57924
   macro avg       0.87      0.87      0.87     57924
weighted avg       0.97      0.97      0.97     57924



In [38]:
print(f"Accuracy = {accuracy_score(y_pred, y):.6f}, \
        Precision (weighted) = {precision_score(y_pred, y, average = 'weighted'):.6f}, \
        Recall (weighted) = {recall_score(y_pred, y, average = 'weighted'):.6f}, \
        F1 (weighted) = {f1_score(y_pred, y, average = 'weighted'):.6f}")

Accuracy = 0.973586,         Precision (weighted) = 0.974048,         Recall (weighted) = 0.973586,         F1 (weighted) = 0.973668


In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
print(confusion_matrix(y_pred, y))

[[ 1926    58    61    12     6     1     5     0    31]
 [   30   689    50    14     0    11     0     0    46]
 [  122    81  2349    57     0     5     9     0    72]
 [    6    21    36  2581     0     0     0     2    34]
 [    9     1     0     0   234     6    18     1    15]
 [    0    13     0     0     3   154     7     1    70]
 [    7     2    33     7    28    20   940    15   114]
 [    1     1     0    23     2     5    13  2479     6]
 [   23   130    59    24     4    39    58     2 45042]]


In [42]:
def print_clarification(idx):
    for sentence, prediction, ground_truth in zip(dataset["test"][idx]["tokens"], true_predictions[idx], true_labels[idx]):
        print(sentence, prediction, ground_truth)

In [43]:
print_clarification(0)


SOCCER O O
- O O
JAPAN B-LOC B-LOC
GET O O
LUCKY O O
WIN O O
, O O
CHINA B-LOC B-PER
IN O O
SURPRISE O O
DEFEAT O O
. O O


In [44]:
print_clarification(10)


Takuya B-PER B-PER
Takagi B-PER B-PER
scored B-PER B-PER
the I-PER I-PER
winner I-PER I-PER
in O O
the O O
88th O O
minute O O
, O O
rising O O
to O O
head O O
a O O
Hiroshige O O
Yanagimoto O O
cross O O
towards O O
the B-PER B-PER
Syrian B-PER B-PER
goal I-PER I-PER
which I-PER I-PER
goalkeeper I-PER I-PER
Salem O O
Bitar O O
appeared O O
to B-MISC B-MISC
have O O
covered O O
but O O
then B-PER B-PER
allowed I-PER I-PER
to I-PER I-PER
slip O O
into O O
the O O
net O O
. O O


In [45]:
print_clarification(20)


Hosts O O
UAE B-LOC B-LOC
play O O
Kuwait B-LOC B-LOC
and O O
South B-LOC B-LOC
Korea I-LOC I-LOC
take O O
on O O
Indonesia B-LOC B-LOC
on O O
Saturday O O
in O O
Group O O
A O O
matches O O
. O O


In [41]:
wandb.finish()

eval/accuracy,▁▅▆▇▇█▇███████
eval/f1,▁▄▆▇▇█▇███████
eval/loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁
eval/precision,▁▄▇▇▇█▇▇██████
eval/recall,▁▅▆▇▇█████████
eval/runtime,▁▁▁▄▂▂▂█▂▂▂▂▂▂
eval/samples_per_second,███▄▇▆▆▁▇▇▇▇▇▆
eval/steps_per_second,███▄▇▆▆▁▇▇▇▇▇▆
train/epoch,▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
train/global_step,▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
train/learning_rate,▁
